# Métrica 1.1.06: "Congruencia entre el nivel jerárquico y tipo de declaración presentada"

In [1]:
import re
import urllib.parse
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure, OperationFailure
import unicodedata
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

# --- 1. CONFIGURACIÓN BÁSICA ---
METRIC_ID = "1_1_06_NIVEL_VS_TIPO_DEC"

# --- 2. LÓGICA PURA DE LA MÉTRICA ---

def normalize_string(s: str) -> str:
    """
    Quita acentos y convierte a mayúsculas para comparar.
    """
    if not isinstance(s, str):
        return ""
    s = ''.join(c for c in unicodedata.normalize('NFD', s)
                if unicodedata.category(c) != 'Mn')
    return s.upper().strip()

def es_jd_o_superior(nivel_str: str) -> int:
    """
    Determina si un string de nivel jerárquico es "Jefe de Departamento" o superior.
    Devuelve:
     1: Si es JD o superior
     0: Si es inferior a JD
    -1: Si el dato es desconocido o no se puede clasificar
    """
    if not isinstance(nivel_str, str) or nivel_str.strip() == "":
        return -1 # Desconocido/Sin Dato
    
    s = normalize_string(nivel_str)
    
    # Lista de rangos INFERIORES a Jefe de Departamento
    # (Todo lo que no esté aquí, se considerará JD o superior)
    rangos_inferiores = [
        "OPERATIVO", 
        "ENLACE", 
        "PERSONAL DE BASE", 
        "SINDICALIZADO"
    ]
    
    if s in rangos_inferiores:
        return 0 # Es inferior a JD
    
    # Lista de rangos SUPERIORES (para estar seguros)
    # Esta lista puede crecer
    rangos_superiores_conocidos = [
        "JEFE DE DEPARTAMENTO", "SUBDIRECTOR DE AREA", 
        "DIRECTOR DE AREA", "DIRECTOR GENERAL", 
        "COORDINADOR GENERAL", "UNIDAD DE MANDO", 
        "SUBSECRETARIO", "SECRETARIO", "PRESIDENTE MUNICIPAL",
        "REGIDOR", "GOBERNADOR", "TITULAR DE UNIDAD"
    ]
    
    if s in rangos_superiores_conocidos:
        return 1 # Es JD o superior
        
    # Si no es un rango inferior conocido, asumimos que es JD o superior
    # por precaución, a menos que queramos marcarlo como desconocido.
    # Para esta métrica, cualquier cosa que no sea explícitamente "inferior"
    # probablemente deba presentar la completa.
    
    # Si no es un rango inferior conocido, lo tratamos como superior.
    return 1

def validar_nivel_vs_tipo_dec(nivel_str: str, es_completa_bool: bool) -> str:
    """
    Valida si un puesto de JD o superior presentó declaración completa.
    """
    
    nivel_num = es_jd_o_superior(nivel_str)
    
    # Caso 1: Faltan datos clave
    if nivel_num == -1 or not isinstance(es_completa_bool, bool):
        return "SIN_DATO"
        
    # Caso 2: El único caso de falla
    # Es JD o superior (nivel 1) Y su declaración NO es completa (False)
    if nivel_num == 1 and not es_completa_bool:
        return "NO_CUMPLE"
        
    # Caso 3: Todos los demás casos cumplen
    # - Es JD o superior Y presentó completa
    # - Es inferior a JD (nivel 0) (no importa cuál presentó)
    return "CUMPLE"

# --- 3. EL FLUJO DEL WORKER (PROCESAMIENTO POR LOTES) ---
def procesar_todas_las_declaraciones_nivel_dec():
    """
    Procesa TODAS las declaraciones de la colección de origen
    y guarda los resultados de la Métrica 1.1.06 en la colección 'metricas'.
    """
    client = None
    
    # Contadores
    total_documentos = 0
    cumple = 0
    no_cumple = 0
    sin_dato = 0
    
    try:
        # --- A. CONECTAR A LA BASE DE DATOS ---
        print(f"Conectando a MongoDB en {DB_NAME}...")
        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        client.admin.command('ping') 
        db = client[DB_NAME]
        
        source_collection = db[SOURCE_COLLECTION_NAME]
        target_collection = db[METRICS_COLLECTION_NAME]
        
        print("¡Conexión exitosa!")
        print(f"Leyendo de: {SOURCE_COLLECTION_NAME}")
        print(f"Escribiendo en: {METRICS_COLLECTION_NAME}")
        print("--- INICIANDO PROCESAMIENTO POR LOTES (Métrica 1.1.06 - Nivel vs Tipo Dec) ---")

        # --- B. BUSCAR TODOS LOS DOCUMENTOS ---
        # Proyección optimizada para traer los 2 campos que necesitamos
        proyeccion = {
            "_id": 1,
            "metadata.declaracionCompleta": 1,
            "declaracion.situacionPatrimonial.datosEmpleoCargoComision.nivelEmpleoCargoComision": 1,
        }
        
        for doc in source_collection.find({}, proyeccion):
            total_documentos += 1
            original_id = doc["_id"]

            # --- C. LEER DATOS Y EJECUTAR MÉTRICA ---
            es_completa = None
            nivel_str = None
            
            try:
                # Navegación segura
                metadata = doc.get("metadata", {})
                if metadata:
                    es_completa = metadata.get("declaracionCompleta") # Debería ser True o False
                
                datos_empleo = doc.get("declaracion", {}).get("situacionPatrimonial", {}).get("datosEmpleoCargoComision", {})
                if datos_empleo:
                    nivel_str = datos_empleo.get("nivelEmpleoCargoComision") # Debería ser un string
                    
            except AttributeError:
                pass # Se queda todo en None

            # Ejecutamos la lógica de la métrica
            resultado_metrica = validar_nivel_vs_tipo_dec(nivel_str, es_completa)
            
            # Actualizamos contadores
            if resultado_metrica == "CUMPLE":
                cumple += 1
            elif resultado_metrica == "NO_CUMPLE":
                no_cumple += 1
            else:
                sin_dato += 1

            # --- CONSERVAMOS TU SALIDA DE CONSOLA ---
            print(f"\nProcesando Documento ID: {original_id}")
            print("--- RESULTADO DEL ANÁLISIS ---")
            print(f"  > Nivel Jerárquico: '{nivel_str}'")
            print(f"  > Declaración Completa: {es_completa}")
            print(f"  > Resultado de la Métrica: {resultado_metrica}")
            print("---------------------------------")
            # --- FIN DE LA SALIDA DE CONSOLA ---

            # --- D. GUARDAR RESULTADO EN LA COLECCIÓN 'metricas' ---
            filtro = { "_id": original_id }
            actualizacion = {
                "$set": {
                    METRIC_ID: resultado_metrica
                }
            }
            target_collection.update_one(filtro, actualizacion, upsert=True)

            print(f"¡Resultado guardado/actualizado en 'metricas' para el ID: {original_id}!")

        print("\n--- PROCESAMIENTO POR LOTES FINALIZADO ---")
        print("Resumen (Métrica 1.1.06 - Nivel vs Tipo Dec):")
        print(f"  > Documentos Totales Procesados: {total_documentos}")
        print(f"  > Métrica 'CUMPLE': {cumple}")
        print(f"  > Métrica 'NO_CUMPLE': {no_cumple}")
        print(f"  > Métrica 'SIN_DATO': {sin_dato}")

    except ConnectionFailure:
        print("Error: No se pudo conectar a la base de datos.")
    except OperationFailure as e:
        print(f"Error en la operación de la base de datos: {e.details}")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
    finally:
        if client:
            client.close()
            print("Conexión cerrada.")

# --- 4. EJECUTAR EL SCRIPT ---
if __name__ == "__main__":
    procesar_todas_las_declaraciones_nivel_dec()

Conectando a MongoDB en sistema1...
¡Conexión exitosa!
Leyendo de: all_data_20251021
Escribiendo en: metricas
--- INICIANDO PROCESAMIENTO POR LOTES (Métrica 1.1.06 - Nivel vs Tipo Dec) ---

Procesando Documento ID: 68f81b8800535f910a29f694
--- RESULTADO DEL ANÁLISIS ---
  > Nivel Jerárquico: 'ASISTENTE'
  > Declaración Completa: True
  > Resultado de la Métrica: CUMPLE
---------------------------------
¡Resultado guardado/actualizado en 'metricas' para el ID: 68f81b8800535f910a29f694!

Procesando Documento ID: 68f81b8800535f910a29f695
--- RESULTADO DEL ANÁLISIS ---
  > Nivel Jerárquico: '0'
  > Declaración Completa: True
  > Resultado de la Métrica: CUMPLE
---------------------------------
¡Resultado guardado/actualizado en 'metricas' para el ID: 68f81b8800535f910a29f695!

Procesando Documento ID: 68f81b8800535f910a29f696
--- RESULTADO DEL ANÁLISIS ---
  > Nivel Jerárquico: '1E0270'
  > Declaración Completa: True
  > Resultado de la Métrica: CUMPLE
---------------------------------
¡Re

KeyboardInterrupt: 